## ThinkStats exercise 1.2

RW: The following solves the problem of old renamed stale variables remaining in the namespace. This clears them. But be careful; it must come first, because it clears everything.
https://github.com/spyder-ide/spyder/issues/2563

In [19]:
from IPython import get_ipython
get_ipython().magic('reset -sf')

In [20]:
# My module for this exercize.
# But I should use the nsfg module for any functions available there.
# No sense copying them! 
import chap01ex
import nsfg

In [21]:
import numpy as np

RW: After much trial and error I believe I've finally solved the problem of not being able to access/see changes made to imported modules. See this link:
https://stackoverflow.com/questions/1254370/reimport-a-module-in-python-while-interactive

In [22]:
import importlib
_ = importlib.reload(chap01ex) # Underscore traps screen output.

#### function ReadFemResp

In [23]:
# Downey provided function
femResp2002 = nsfg.ReadFemResp()
femResp2002.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7643 entries, 0 to 7642
Columns: 3087 entries, caseid to intvlngth
dtypes: float64(2491), int64(595), object(1)
memory usage: 180.0+ MB


In [24]:
cols = femResp2002.columns
print(type(cols))
print(len(cols))
print(femResp2002.index)
femResp2002.columns

<class 'pandas.core.indexes.base.Index'>
3087
RangeIndex(start=0, stop=7643, step=1)


Index(['caseid', 'rscrinf', 'rdormres', 'rostscrn', 'rscreenhisp',
       'rscreenrace', 'age_a', 'age_r', 'cmbirth', 'agescrn',
       ...
       'pubassis_i', 'basewgt', 'adj_mod_basewgt', 'finalwgt', 'secu_r',
       'sest', 'cmintvw', 'cmlstyr', 'screentime', 'intvlngth'],
      dtype='object', length=3087)

#### variable pregnum
\>>>The variable pregnum is a recode that indicates how many times each respondent has been pregnant.<<<

But note that this variable exists in two separate files, 2002FemResp and 2002FemPreg.

In [25]:
pregnum_fem_resp = femResp2002.pregnum

In [26]:
print(len(pregnum_fem_resp))
print(type(pregnum_fem_resp))
print(pregnum_fem_resp[:5])

7643
<class 'pandas.core.series.Series'>
0    4
1    1
2    1
3    0
4    0
Name: pregnum, dtype: int64


##### fem resp pregnum value counts

In [27]:
prgnm_fem_resp_val_cnts = pregnum_fem_resp.value_counts().sort_index()
print(type(prgnm_fem_resp_val_cnts))
prgnm_fem_resp_val_cnts

<class 'pandas.core.series.Series'>


0     2610
1     1267
2     1432
3     1110
4      611
5      305
6      150
7       80
8       40
9       21
10       9
11       3
12       2
14       2
19       1
Name: pregnum, dtype: int64

In [28]:
prgnm_fem_resp_val_cnts.sum()

7643

The above total matches the Female Respondent File codebook as do the individual counts below.

In [29]:
# Check to verify index is number of pregnancies
prgnm_fem_resp_val_cnts.loc[19]

1

In [30]:
pg_svn_or_more = prgnm_fem_resp_val_cnts.loc[7:]
print(pg_svn_or_more)
pg_svn_or_more.sum()

7     80
8     40
9     21
10     9
11     3
12     2
14     2
19     1
Name: pregnum, dtype: int64


158

#### cross-validation on number of pregnancies per caseid
You can...cross-validate the respondent (2002FemResp) and pregnancy (2002FemPreg) files by comparing pregnum for each respondent with the number of records in the pregnancy file.

In [31]:
# This was previously established in chap01ex.ipynb
preg = nsfg.ReadFemPreg() # 2002FemPreg
preg.head()

,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,multbrth,...,laborfor_i,religion_i,metro_i,basewgt,adj_mod_basewgt,finalwgt,secu_p,sest,cmintvw,totalwgt_lb
0,1,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,3410.389399,3869.349602,6448.271112,2,9,NaN,8.8125
1,1,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,3410.389399,3869.349602,6448.271112,2,9,NaN,7.8750
2,2,1,NaN,NaN,NaN,NaN,5.0,NaN,3.0,5.0,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,9.1250
3,2,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,7.0000
4,2,3,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,6.1875


In [32]:
caseid = 10229

##### 2002FemPreg

In [33]:
preg_map = nsfg.MakePregMap(preg) # 2002FemPreg
indices = preg_map[caseid]
print(preg.outcome[indices].values) # example from chap01ex.ipynb
print(len(preg.loc[indices]))
print(preg.loc[indices].shape[0])

[4 4 4 4 4 4 1]
7
7


##### 2002FemResp

In [34]:
femResp2002[femResp2002['caseid'] == caseid][['caseid', 'pregnum']] # 2002FemResp

,caseid,pregnum
3237,10229,7


##### Using Downey's solution

In [35]:
result = chap01ex.ValidatePregnum(femResp2002)
print(result)
if(result):
    print("pregnum data cross-validated")
else: print("pregnum data fails cross-validation")

True
pregnum data cross-validated
